<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week06/Python_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Hands-on : Database with python
### 1. superbase에 가입하고 프로젝트를 생성한 후
### 2. sqlite3로 DB를 만들고 "movie" 데이터 생성
### 3. 엑셀로 다운받고
### 4. superbase에서 저장하기

### **1. [supabase](https://supabase.com/) 클라우드 데이터베이스 만들기**
- https://supabase.com/dashboard/project/gjuciogiegqiimposzkc

In [1]:
!pip install supabase python-dotenv -q

In [2]:
from google.colab import userdata
from supabase import create_client, Client

# Colab Secrets에서 URL과 Key를 가져옵니다.
# Colab Secret Manager를 사용하면 os.environ 대신 userdata.get()을 사용합니다.
supabase_key: str = userdata.get('superbase')

# Supabase URL 및 Key 설정 (JavaScript 코드와 동일한 값 사용)
supabase_url: str = 'https://gjuciogiegqiimposzkc.supabase.co'

# Supabase 클라이언트 생성
if supabase_url and supabase_key:
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase 클라이언트가 성공적으로 연결되었습니다.")
else:
    print("⚠️ Colab Secrets에서 SUPABASE_URL 또는 SUPABASE_KEY를 찾을 수 없습니다. Secret 설정을 확인해주세요.")

✅ Supabase 클라이언트가 성공적으로 연결되었습니다.


### superbase에 테이블을 만들지 않아서 No Tables

### **2. [sqlite3 — DB-API 2.0 interface for SQLite databases](https://docs.python.org/3/library/sqlite3.html)**

In [3]:
import sqlite3
con = sqlite3.connect("tutorial.db")

In [4]:
cur = con.cursor()

In [5]:
cur.execute("CREATE TABLE movie(title, year, score)")

In [6]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie',)

In [7]:
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
res.fetchone() is None

True

In [8]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

In [9]:
con.commit()

In [10]:
res = cur.execute("SELECT score FROM movie")
res.fetchall()

[(8.2,), (7.5,)]

In [11]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()  # Remember to commit the transaction after executing INSERT.

In [12]:
for row in cur.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)

(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


In [13]:
con.close()
new_con = sqlite3.connect("tutorial.db")
new_cur = new_con.cursor()
res = new_cur.execute("SELECT title, year FROM movie ORDER BY score DESC")
title, year = res.fetchone()
print(f'The highest scoring Monty Python movie is {title!r}, released in {year}')

new_con.close()

The highest scoring Monty Python movie is 'Monty Python and the Holy Grail', released in 1975


### **3. 데이터프레임으로 다운받고 엑셀로 내보내기**

In [14]:
import pandas as pd
# Connect to the database
con = sqlite3.connect("tutorial.db")

# Read the table into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM movie", con)

# Display the DataFrame
display(df)

# Close the connection
con.close()

,title,year,score
0,Monty Python and the Holy Grail,1975,8.2
1,And Now for Something Completely Different,1971,7.5
2,Monty Python Live at the Hollywood Bowl,1982,7.9
3,Monty Python's The Meaning of Life,1983,7.5
4,Monty Python's Life of Brian,1979,8.0


In [15]:
df.to_excel("movie.xlsx")

### **4. superbase로 upload하고 확인하기**

In [16]:

# # Colab Secrets에서 URL과 Key를 가져옵니다.
# supabase_url: str = 'https://gjuciogiegqiimposzkc.supabase.co'
# supabase_key: str = userdata.get('supabase') # Use the correct secret name 'supabase'

# Supabase 클라이언트 생성
if supabase_url and supabase_key:
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase 클라이언트가 성공적으로 연결되었습니다.")

    # DataFrame을 Supabase에 삽입
    # 'movie'는 Supabase에 생성될 테이블 이름입니다.
    # upsert=True는 충돌 발생 시 업데이트하도록 설정합니다.
    try:
        # Convert DataFrame to a list of dictionaries for Supabase insertion
        data_to_insert = df.to_dict('records')

        response = supabase.table("movie").insert(data_to_insert).execute()

        if response:
            print("✅ 데이터가 Supabase에 성공적으로 업로드되었습니다.")
            # print("응답:", response) # Uncomment to see the response

        else:
            print("⚠️ 데이터 업로드에 실패했습니다.")
            # print("응답:", response) # Uncomment to see the response


    except Exception as e:
        print(f"❌ 데이터 업로드 중 오류가 발생했습니다: {e}")

else:
    print("⚠️ Colab Secrets에서 SUPABASE_URL 또는 SUPABASE_KEY를 찾을 수 없습니다. Secret 설정을 확인해주세요.")

✅ Supabase 클라이언트가 성공적으로 연결되었습니다.
✅ 데이터가 Supabase에 성공적으로 업로드되었습니다.


In [17]:
# DataFrame을 JSON으로 변환 (orient='records'로 리스트 형태 생성)
movies_data = df.to_dict(orient='records')
movies_data

[{'title': 'Monty Python and the Holy Grail', 'year': 1975, 'score': 8.2},
 {'title': 'And Now for Something Completely Different',
  'year': 1971,
  'score': 7.5},
 {'title': 'Monty Python Live at the Hollywood Bowl',
  'year': 1982,
  'score': 7.9},
 {'title': "Monty Python's The Meaning of Life", 'year': 1983, 'score': 7.5},
 {'title': "Monty Python's Life of Brian", 'year': 1979, 'score': 8.0}]

#### **먼저 Supabase에서 "movie" 테이블을 생성해야 함**


In [18]:
'''
CREATE TABLE movie (
  id BIGSERIAL PRIMARY KEY,
  title TEXT NOT NULL,
  year INTEGER NOT NULL,
  score DOUBLE PRECISION NOT NULL,
  created_at TIMESTAMP WITH TIME ZONE DEFAULT NOW()
);

-- RLS 비활성화 (개발/테스트용)
ALTER TABLE movie DISABLE ROW LEVEL SECURITY;
'''

'\nCREATE TABLE movie (\n  id BIGSERIAL PRIMARY KEY,\n  title TEXT NOT NULL,\n  year INTEGER NOT NULL,\n  score DOUBLE PRECISION NOT NULL,\n  created_at TIMESTAMP WITH TIME ZONE DEFAULT NOW()\n);\n\n-- RLS 비활성화 (개발/테스트용)\nALTER TABLE movie DISABLE ROW LEVEL SECURITY;\n'

In [19]:
# # Supabase 연결 설정
# supabase_key: str = userdata.get('superbase')
# supabase_url: str = 'https://gjuciogiegqiimposzkc.supabase.co'

# Supabase 클라이언트 생성
if supabase_url and supabase_key:
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase 클라이언트가 성공적으로 연결되었습니다.")
else:
    print("⚠️ Colab Secrets에서 SUPABASE_URL 또는 SUPABASE_KEY를 찾을 수 없습니다.")
    exit()

# DataFrame을 JSON으로 변환 (orient='records'로 리스트 형태 생성)
movies_data = df.to_dict(orient='records')

print("📝 JSON 형태로 변환된 데이터:")
print(movies_data)
print()

try:
    # 데이터 삽입
    print("📤 영화 데이터를 Supabase에 업로드 중...")
    response = supabase.table('movie').insert(movies_data).execute()

    print(f"✅ {len(movies_data)}개의 영화 데이터가 성공적으로 저장되었습니다!")
    print("\n저장된 데이터:")

    # 저장된 데이터 확인
    result = supabase.table('movie').select("*").execute()
    result_df = pd.DataFrame(result.data)
    print(result_df.to_string(index=False))

except Exception as e:
    print(f"❌ 오류 발생: {str(e)}")

✅ Supabase 클라이언트가 성공적으로 연결되었습니다.
📝 JSON 형태로 변환된 데이터:
[{'title': 'Monty Python and the Holy Grail', 'year': 1975, 'score': 8.2}, {'title': 'And Now for Something Completely Different', 'year': 1971, 'score': 7.5}, {'title': 'Monty Python Live at the Hollywood Bowl', 'year': 1982, 'score': 7.9}, {'title': "Monty Python's The Meaning of Life", 'year': 1983, 'score': 7.5}, {'title': "Monty Python's Life of Brian", 'year': 1979, 'score': 8.0}]

📤 영화 데이터를 Supabase에 업로드 중...
✅ 5개의 영화 데이터가 성공적으로 저장되었습니다!

저장된 데이터:
 id                                      title  year  score                       created_at
  1            Monty Python and the Holy Grail  1975    8.2 2025-10-11T15:43:59.023636+00:00
  2 And Now for Something Completely Different  1971    7.5 2025-10-11T15:43:59.023636+00:00
  3    Monty Python Live at the Hollywood Bowl  1982    7.9 2025-10-11T15:43:59.023636+00:00
  4         Monty Python's The Meaning of Life  1983    7.5 2025-10-11T15:43:59.023636+00:00
  5               Monty